In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.integrate import odeint, solve_ivp
from matplotlib.animation import FuncAnimation
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap


plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.5)
colors = sns.color_palette("viridis", 4)


def heatwave_model(y, t, params, weather_data=None):
    """
    Enhanced SHRD model (Susceptible-Hospitalized-Recovered-Deceased) for heat wave impacts
    """
    S, H, R, D = y


    beta_base, gamma_base, mu_base, alpha_T, alpha_P, alpha_V, alpha_A, alpha_H, hospital_threshold = params


    if weather_data is None:
        T = 60  # Default temperature (°C)
        P = 150  # Default pollution level (PM2.5)
        H_rel = 60  # Default humidity (%)
        V = 0.5  # Default vulnerability index
        A = 0.3  # Default awareness level
        C = 200  # Default hospital capacity
        U = 5  # Default urban heat island effect
    else:

        T = np.interp(t, weather_data['time'], weather_data['temperature'])
        P = np.interp(t, weather_data['time'], weather_data['pollution'])
        H_rel = np.interp(t, weather_data['time'], weather_data['humidity'])


        V = weather_data['vulnerability'] * np.exp(-0.02 * t)


        A = min(0.9, weather_data['awareness'] + 0.01 * t * max(0, T - 35))


        C = weather_data['hospital_capacity']


        U = weather_data['urban_heat_island']


    utilization = H / C if C > 0 else 1.0


    temperature_effect = alpha_T * np.tanh(max(0, (T - 35)) / 5)


    pollution_effect = alpha_P * np.tanh(P / 100)


    vulnerability_effect = alpha_V * V


    awareness_effect = -alpha_A * A


    beta_eff = beta_base * (1 + temperature_effect + pollution_effect + vulnerability_effect + awareness_effect)
    beta_eff = max(0, beta_eff)  # Ensure non-negative

    # Effective recovery rate (decreases when hospitals are overwhelmed)
    gamma_eff = gamma_base * (1 - 0.5 * np.tanh(max(0, utilization - hospital_threshold)))
    gamma_eff = max(0.01, gamma_eff)  # Ensure positive recovery rate

    # Effective mortality rate (increases with humidity and hospital overcrowding)
    humidity_effect = alpha_H * (H_rel / 100)
    overcrowding_effect = np.exp(max(0, utilization - hospital_threshold)) - 1

    mu_eff = mu_base * (1 + humidity_effect + overcrowding_effect)

    # System dynamics
    dSdt = -beta_eff * S
    dHdt = beta_eff * S - (gamma_eff + mu_eff) * H
    dRdt = gamma_eff * H
    dDdt = mu_eff * H

    return [dSdt, dHdt, dRdt, dDdt]

# Function to generate synthetic weather data for simulation
def generate_synthetic_weather(duration=30, n_points=100, base_temp=35, temp_amplitude=10,
                             base_pollution=80, pollution_amplitude=40):
    """Generate synthetic weather data for simulation"""
    time = np.linspace(0, duration, n_points)

    # Generate temperature with a realistic pattern (daily cycle + trend)
    temperature = base_temp + 5 * np.sin(2 * np.pi * time / 1) + temp_amplitude * np.sin(2 * np.pi * time / duration)

    # Generate pollution data (correlated with temperature but with some randomness)
    pollution = base_pollution + pollution_amplitude * 0.7 * np.sin(2 * np.pi * time / duration) + \
               pollution_amplitude * 0.3 * np.random.randn(n_points)
    pollution = np.maximum(0, pollution)  # Ensure non-negative

    # Generate humidity data (partially anti-correlated with temperature)
    humidity = 50 - 20 * np.sin(2 * np.pi * time / duration) + 10 * np.random.randn(n_points)
    humidity = np.clip(humidity, 10, 90)  # Keep within realistic bounds

    return {
        'time': time,
        'temperature': temperature,
        'pollution': pollution,
        'humidity': humidity,
        'vulnerability': 0.5,  # Initial vulnerability index
        'awareness': 0.2,      # Initial awareness level
        'hospital_capacity': 500,  # Hospital capacity
        'urban_heat_island': 3.0   # Urban heat island effect
    }

# Function for simulation (renamed to match usage)
def adaptive_integration(model, initial_conditions, time_points, params, weather_data=None):
    """
    Renamed from simulate_model to match usage in other functions
    """
    if weather_data is None:
        # Use simple odeint for constant parameter case
        solution = odeint(model, initial_conditions, time_points, args=(params,))
        return time_points, solution.T
    else:
        # Use solve_ivp for more complex case with time-dependent parameters
        solution = solve_ivp(
            lambda t, y: model(y, t, params, weather_data),
            t_span=[time_points[0], time_points[-1]],
            y0=initial_conditions,
            method='RK45',  # Runge-Kutta 4(5) with adaptive step size
            t_eval=time_points,
            rtol=1e-6,
            atol=1e-6
        )
        return solution.t, solution.y

# Function to model intervention strategies
def model_interventions(baseline_params, weather_data, interventions, duration=30, n_points=100):
    """Model different intervention strategies"""
    results = {}
    time_points = np.linspace(0, duration, n_points)
    initial_conditions = [10000, 100, 0, 0]  # [S0, H0, R0, D0]

    # Baseline scenario (no intervention)
    _, solution = adaptive_integration(
        heatwave_model,
        initial_conditions,
        time_points,
        baseline_params,
        weather_data
    )

    results['Baseline'] = {
        'time': time_points,
        'S': solution[0],
        'H': solution[1],
        'R': solution[2],
        'D': solution[3]
    }

    # Simulate each intervention
    for name, intervention in interventions.items():
        # Create a modified weather_data dictionary for this intervention
        modified_data = weather_data.copy()

        # Apply intervention effects
        if 'awareness_boost' in intervention:
            modified_data['awareness'] = min(1.0, weather_data['awareness'] + intervention['awareness_boost'])

        if 'hospital_capacity_increase' in intervention:
            modified_data['hospital_capacity'] = weather_data['hospital_capacity'] + intervention['hospital_capacity_increase']

        if 'vulnerability_reduction' in intervention:
            modified_data['vulnerability'] = max(0.05, weather_data['vulnerability'] * (1 - intervention['vulnerability_reduction']))

        # Run simulation with modified data
        _, solution = adaptive_integration(
            heatwave_model,
            initial_conditions,
            time_points,
            baseline_params,
            modified_data
        )

        results[name] = {
            'time': time_points,
            'S': solution[0],
            'H': solution[1],
            'R': solution[2],
            'D': solution[3]
        }

    return results

# Function to evaluate intervention effectiveness
def evaluate_interventions(results):
    """Calculate metrics to evaluate intervention effectiveness"""
    metrics = []

    for name, data in results.items():
        metrics.append({
            'Intervention': name,
            'Peak Hospital Admissions': np.max(data['H']),
            'Time to Peak (days)': data['time'][np.argmax(data['H'])],
            'Total Deaths': data['D'][-1],
            'Final Susceptible': data['S'][-1],
            'Hospital Burden Duration': np.sum(data['H'] > 0.5 * np.max(data['H'])) * (data['time'][1] - data['time'][0]),
            'Cost Effectiveness': 1.0  # Placeholder - would be calculated based on intervention cost
        })

    return pd.DataFrame(metrics)

# 1. Multi-panel Dashboard for Comprehensive Analysis
def create_dashboard(results, figsize=(20, 16)):
    """Create a comprehensive multi-panel dashboard visualization."""
    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(3, 3, figure=fig)

    # Panel 1: Hospital Admissions Time Series
    ax1 = fig.add_subplot(gs[0, :])
    for scenario, data in results.items():
        ax1.plot(data['time'], data['H'], label=scenario, linewidth=2.5)
    ax1.set_ylabel('Hospital Admissions')
    ax1.set_title('Impact of Interventions on Hospital Admissions', fontsize=16)
    ax1.legend(loc='upper right')

    # Panel 2: Mortality Time Series
    ax2 = fig.add_subplot(gs[1, :])
    for scenario, data in results.items():
        ax2.plot(data['time'], data['D'], label=scenario, linewidth=2.5, linestyle='--')
    ax2.set_ylabel('Cumulative Deaths')
    ax2.set_title('Impact of Interventions on Mortality', fontsize=16)

    # Panel 3: Peak Hospital Burden
    ax3 = fig.add_subplot(gs[2, 0])
    scenarios = list(results.keys())
    max_admissions = [np.max(results[s]['H']) for s in scenarios]
    ax3.bar(scenarios, max_admissions, color=colors)
    ax3.set_ylabel('Peak Hospital Admissions')
    ax3.set_title('Maximum Hospital Burden', fontsize=14)
    plt.setp(ax3.get_xticklabels(), rotation=45, ha='right')

    # Panel 4: Final Death Toll
    ax4 = fig.add_subplot(gs[2, 1])
    final_deaths = [results[s]['D'][-1] for s in scenarios]
    ax4.bar(scenarios, final_deaths, color=colors)
    ax4.set_ylabel('Total Deaths')
    ax4.set_title('Final Death Toll', fontsize=14)
    plt.setp(ax4.get_xticklabels(), rotation=45, ha='right')

    # Panel 5: Recovery Dynamics
    ax5 = fig.add_subplot(gs[2, 2])
    for scenario, data in results.items():
        recovery_rate = np.diff(data['R']) / np.diff(data['time'])
        ax5.plot(data['time'][1:], recovery_rate, label=scenario)
    ax5.set_ylabel('Recovery Rate (per day)')
    ax5.set_xlabel('Days')
    ax5.set_title('Recovery Dynamics', fontsize=14)

    plt.tight_layout()
    return fig

# 3. Animated Time Evolution Visualization
def create_animation(results, scenario_names=None, interval=200):
    """Create an animated visualization showing the time evolution of the model."""
    if scenario_names is None:
        scenario_names = list(results.keys())

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

    max_H = max([np.max(results[s]['H']) for s in scenario_names])
    max_D = max([np.max(results[s]['D']) for s in scenario_names])

    lines_H = []
    lines_D = []

    # Setup axes
    ax1.set_xlim(0, results[scenario_names[0]]['time'][-1])
    ax1.set_ylim(0, max_H * 1.1)
    ax1.set_ylabel('Hospital Admissions')
    ax1.set_title('Hospital Admissions Over Time')

    ax2.set_xlim(0, results[scenario_names[0]]['time'][-1])
    ax2.set_ylim(0, max_D * 1.1)
    ax2.set_xlabel('Days')
    ax2.set_ylabel('Cumulative Deaths')
    ax2.set_title('Mortality Over Time')

    # Initialize lines
    for i, scenario in enumerate(scenario_names):
        line_H, = ax1.plot([], [], label=scenario, linewidth=2)
        line_D, = ax2.plot([], [], label=scenario, linewidth=2, linestyle='--')
        lines_H.append(line_H)
        lines_D.append(line_D)

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper left')

    def init():
        for line_H, line_D in zip(lines_H, lines_D):
            line_H.set_data([], [])
            line_D.set_data([], [])
        return lines_H + lines_D

    def update(frame):
        for i, scenario in enumerate(scenario_names):
            time_data = results[scenario]['time'][:frame]
            H_data = results[scenario]['H'][:frame]
            D_data = results[scenario]['D'][:frame]

            lines_H[i].set_data(time_data, H_data)
            lines_D[i].set_data(time_data, D_data)

        return lines_H + lines_D

    ani = FuncAnimation(fig, update, frames=len(results[scenario_names[0]]['time']),
                        init_func=init, blit=True, interval=interval)
    plt.tight_layout()

    return ani

# 7. Scenario Comparison Radar Chart
def radar_chart(results, figsize=(10, 10)):
    """Create a radar chart to compare different scenarios across multiple metrics."""
    scenario_names = list(results.keys())

    # Calculate normalized metrics for each scenario
    metrics = {
        'Peak Hospital': [np.max(results[s]['H']) for s in scenario_names],
        'Total Deaths': [results[s]['D'][-1] for s in scenario_names],
        'Recovery Rate': [np.mean(np.diff(results[s]['R'])) for s in scenario_names],
        'Time to Peak': [results[s]['time'][np.argmax(results[s]['H'])] for s in scenario_names],
        'Final Susceptible': [results[s]['S'][-1] for s in scenario_names]
    }

    # Normalize metrics for radar chart
    for metric in metrics:
        max_val = max(metrics[metric])
        if max_val > 0:  # Avoid division by zero
            metrics[metric] = [val/max_val for val in metrics[metric]]

    # Compute angles for radar chart
    metric_names = list(metrics.keys())
    angles = np.linspace(0, 2*np.pi, len(metric_names), endpoint=False).tolist()
    angles += angles[:1]  # Close the circle

    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(polar=True))

    for i, scenario in enumerate(scenario_names):
        values = [metrics[m][i] for m in metric_names]
        values += values[:1]  # Close the circle

        ax.plot(angles, values, linewidth=2, label=scenario)
        ax.fill(angles, values, alpha=0.1)

    # Set category labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metric_names)

    # Add legend
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.title('Intervention Scenario Comparison', fontsize=16)
    return fig

# Run a complete simulation and generate visualizations
def run_complete_simulation():
    # Generate synthetic weather data
    weather_data = generate_synthetic_weather(duration=30)

    # Define model parameters
    # (beta_base, gamma_base, mu_base, alpha_T, alpha_P, alpha_V, alpha_A, alpha_H, hospital_threshold)
    baseline_params = (0.0001, 0.2, 0.01, 0.8, 0.3, 0.5, 0.6, 0.4, 0.7)

    # Define intervention strategies
    interventions = {
        'Public Awareness Campaign': {
            'awareness_boost': 0.3
        },
        'Hospital Capacity Expansion': {
            'hospital_capacity_increase': 200
        },
        'Vulnerability Reduction': {
            'vulnerability_reduction': 0.3
        },
        'Combined Strategy': {
            'awareness_boost': 0.3,
            'hospital_capacity_increase': 200,
            'vulnerability_reduction': 0.3
        }
    }

    # Run simulations
    results = model_interventions(baseline_params, weather_data, interventions, duration=30)

    # Evaluate interventions
    evaluation = evaluate_interventions(results)
    print("Intervention Evaluation:")
    print(evaluation)

    # Generate visualizations
    dashboard = create_dashboard(results)
    plt.savefig('heatwave_dashboard.png')
    plt.close()

    # Create radar chart
    radar = radar_chart(results)
    plt.savefig('intervention_radar_chart.png')
    plt.close()

    # Create animation (for animation, we'll just show a single frame for simplicity)
    print("Animation would be created here in a running environment.")

    return results, evaluation

# When this script is run directly, execute the simulation
if __name__ == "__main__":
    results, evaluation = run_complete_simulation()

Intervention Evaluation:
                  Intervention  Peak Hospital Admissions  Time to Peak (days)  \
0                     Baseline                     100.0                  0.0   
1    Public Awareness Campaign                     100.0                  0.0   
2  Hospital Capacity Expansion                     100.0                  0.0   
3      Vulnerability Reduction                     100.0                  0.0   
4            Combined Strategy                     100.0                  0.0   

   Total Deaths  Final Susceptible  Hospital Burden Duration  \
0      7.517270        9957.162848                  3.939394   
1      7.322295        9961.379552                  3.939394   
2      7.517270        9957.162848                  3.939394   
3      7.434902        9958.844183                  3.939394   
4      7.239132        9963.075726                  3.939394   

   Cost Effectiveness  
0                 1.0  
1                 1.0  
2                 1.0  
3      